In [ ]:

import pandas as pd
from io import StringIO
import imp
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import sys
import requests
import json
import os
import time
from datetime import datetime
import logging

logging.getLogger().setLevel(logging.INFO)

sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/lending_recon/src")
import utils
sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/lending_recon/src/vault_utils.py")
import vault_utils

imp.reload(utils)
imp.reload(vault_utils)


# import vault_utils
def get_ads_balances():
    ## Anchorage Vault Utility
    avu = vault_utils.AnchorageVaultUtility(
        api_key_str,
        signing_key_str
    )
    data = avu.query_all_vaults()

    flattened_data = []
    for account in data:
        account_name = account['accountName']
        for asset in account['assets']:
            flattened_data.append({
                'accountName': account_name,
                'assetType': asset['assetType'],
                'availableBalance_currentPrice': asset['availableBalance']['currentPrice'],
                'availableBalance_currentUSDValue': asset['availableBalance']['currentUSDValue'],
                'availableBalance_quantity': asset['availableBalance']['quantity'],
                'totalBalance_currentPrice': asset['totalBalance']['currentPrice'],
                'totalBalance_currentUSDValue': asset['totalBalance']['currentUSDValue'],
                'totalBalance_quantity': asset['totalBalance']['quantity'],
                'vaultId': asset['vaultId'],
                'vaultName': asset['vaultName'],
                'walletId': asset['walletId']
            })

    # Creating the DataFrame
    df = pd.DataFrame(flattened_data)
    return df

sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/quant_lib/anchoragequantlib")
import utils as aql_utils

# Load params
os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Mock A1 Risk Dashboard"
tab_name = "ads_balances"

ads_balances = get_ads_balances()
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name, ads_balances)



/tmp/ipykernel_283130/4073470784.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
INFO:root:getting vaults path: https://api.anchorage.com/v2/vaults?limit=100


SpreadsheetNotFound: <Response [200]>